# Setup
**This notebook serves as a demo and tutorial for the yielengine package.**
It illustrates the following methods:
- Usage of the yieldengine API that wrap sklearn functionalities so that transformes output dataframe with meaningfull column names
- EDA
- Boruta feature selection
- Shap clustering feature selection
- Simulaiton

Possible datasets:
- https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength concrete strength (9 variables)
- https://archive.ics.uci.edu/ml/datasets/Energy+efficiency energetical performance (8 variables)
- https://archive.ics.uci.edu/ml/datasets/Insurance+Company+Benchmark+%28COIL+2000%29 : regression problem: know if a client subscribes for an insurance
- https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity Popularity of an article

In [ ]:
# Print the used verion of Python to be sure that you are using the yield-engine environement
import sys
print(sys.executable)

In [ ]:
PATH_YIELD_ENGINE = 'src'

def set_paths() -> None:
    """
    set correct working directory and python path when started from within PyCharm
    """
    import sys
    import os
    
    if 'cwd' not in globals():
        # noinspection PyGlobalUndefined
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir)
        os.chdir(cwd)
    
    print(f"working dir is '{os.getcwd()}'")
                             
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    
    print(f"added `{sys.path[0]}` to python paths")

set_paths()

In [ ]:
import logging
import os    
import re
import pprint
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error
from typing import *

from yieldengine import Sample
from yieldengine.dendrogram import DendrogramDrawer
from yieldengine.dendrogram.style import FeatMapStyle, LineStyle
from yieldengine.df.pipeline import PipelineDF
from yieldengine.preprocessing.encode import OneHotEncoderDF
from yieldengine.preprocessing.impute import SimpleImputerDF, MissingIndicatorDF
from yieldengine.preprocessing.selection import BorutaDF
from yieldengine.preprocessing.compose import ColumnTransformerDF
from yieldengine.model.inspection import ModelInspector
from yieldengine.model.prediction import PredictorFitCV
from yieldengine.model.selection import ModelPipelineDF, ModelGrid, ModelRanker, summary_report, ModelEvaluation, ModelScoring
from yieldengine.model.validation import CircularCrossValidator
from yieldengine.visualization.eda import plot_ecdf, plot_ecdf_df, plot_hist_df
from yieldengine.preprocessing.outlier import OutlierRemoverDF
from yieldengine.preprocessing import FunctionTransformerDF
from yieldengine.simulation import UnivariateSimulator
from yieldengine.partition import ContinuousRangePartitioning

In [ ]:
# Enlarge the width of the cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Only show matplolib warning logging messages, otherwise there are too many
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)
log = logging.getLogger(__name__)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 8]

In [ ]:
IQR_MULTIPLE = 1.5

# Load raw data, EDA and data preprocessing

## Raw data

In [ ]:
DATA_DIR = os.path.join('data', 'ames-housing-dataset')
RAW_DATA_CSV = os.path.join(DATA_DIR, 'train.csv')
TARGET = "SalePrice"

print(f"Read {RAW_DATA_CSV}")
raw_df = pd.read_csv(RAW_DATA_CSV)

## EDA

In [ ]:
print(f"Shape of the raw data: {raw_df.shape}")
raw_df.head()

In [ ]:
raw_df.columns

## Data Prep

In [ ]:
dataset_df = raw_df.copy()
# Drop the Id column and date columns
dataset_df = dataset_df.drop(['Id', 'YrSold', 'MoSold'], axis=1)

In [ ]:
# remove infinity values
dataset_df = dataset_df.replace([np.inf, -np.inf], np.nan)

In [ ]:
features_df = dataset_df.drop(columns=TARGET)
cat_features = features_df.select_dtypes(['category', object]).columns
num_features = features_df.select_dtypes('number').columns
print (f"{features_df.shape[1] - len(cat_features) - len(num_features)} not a number or category")
del features_df

for col_name in cat_features:
    dataset_df.loc[:, col_name] = (
        dataset_df.loc[:, col_name].astype(object).fillna('missing value').astype(str).str.strip().str.lower().astype('category')
    )
# output datasets
dt = dataset_df.dtypes.rename('dtype').astype(str)
dt.groupby(by=dt.values).count().to_frame()

For simplicity we select only numerical features

In [ ]:
dataset_df = dataset_df[list(num_features) + [TARGET]]

## Remove outliers
We remove the samples whose target value is an outlier

In [ ]:
outlier_remover = OutlierRemoverDF(iqr_multiple=IQR_MULTIPLE)

In [ ]:
# Filter out the rows of the dataset where the target is  an outlier
print(f"Shape before removal of target outlier: {dataset_df.shape}")
mask = outlier_remover.fit_transform(dataset_df[[TARGET]])[TARGET].notna()
dataset_df = dataset_df.loc[mask, :]
print(f"Shape after removal of target outlier: {dataset_df.shape}")

## Create Sample object to be used by our model
The **Sample** class is a wrapper around a dataframe, which also specifies which columns are features and which column is the target.

In [ ]:
# Define a Sample object to be used by our model
sample_full = Sample(
    observations=dataset_df,
    target_name=TARGET)

* Plot of the empirical cumulative distribution function (ECDF) of the target 

In [ ]:
plot_ecdf(sample_full.target, iqr_multiple=IQR_MULTIPLE, iqr_multiple_far=IQR_MULTIPLE*2)

# Boruta feature selection
In a nutshell, the Boruta algorithm discards the features which are not more predictive than random noise. 
See https://www.datacamp.com/community/tutorials/feature-selection-R-boruta  for more explanations.

## imputation and encoding
We first define a Pipeline that will be used before running Boruta. All the transformers are wrapper around scikit-learn transformers which return a Dataframe with the appropriate column names.

In [ ]:
def make_preprocessor(sample: Sample, onehot: bool = True, missing: bool = True) -> ColumnTransformerDF:
    tx = []
    # define how features should be preprocessed
    tx.append(("impute", 
               SimpleImputerDF(strategy="median"), 
               sample.features_by_type(Sample.DTYPE_NUMERICAL).columns))

    if missing:
        tx.append(("missing", 
                   MissingIndicatorDF(error_on_new=False), 
                   sample.features_by_type(Sample.DTYPE_NUMERICAL).columns))
        
    if onehot:
        tx.append(("onehot", 
                   OneHotEncoderDF(sparse=False, handle_unknown="ignore"),
                   sample.features_by_type([Sample.DTYPE_CATEGORICAL]).columns))     

    return ColumnTransformerDF(transformers=tx)

def make_outlier_transformer(sample: Sample, iqr_threshold) -> ColumnTransformerDF:    
    outlier_transformers = [
        ('outlier', OutlierRemoverDF(iqr_multiple=IQR_MULTIPLE), sample_full.features_by_type(Sample.DTYPE_NUMERICAL).columns),
        ('rest', FunctionTransformerDF(validate=False), sample_full.features_by_type(Sample.DTYPE_OBJECT).columns)
    ]
    outlier_step = ColumnTransformerDF(transformers=outlier_transformers)
    return outlier_step

In [ ]:
preprocessor = make_preprocessor(sample=sample_full, missing=True, onehot=True)
outlier_step = make_outlier_transformer(sample=sample_full, iqr_threshold=IQR_MULTIPLE)

## Feature selection (Boruta)

In [ ]:
boruta_selector = PipelineDF(
    steps = [
        ('outlier_removal', outlier_step),
        ('preprocess', preprocessor),
        ('boruta', BorutaDF(estimator=RandomForestRegressor(max_depth=5,min_samples_leaf=8,
                                                            random_state=42,n_jobs=-3),
                             n_estimators=10, verbose=2, max_iter=10, random_state=42))])

In [ ]:
boruta_selector.fit(sample_full.features, sample_full.target)

In [ ]:
features_removed_by_boruta = set(boruta_selector.columns_in) - set(boruta_selector.columns_original)
print(f"Boruta remove {len(features_removed_by_boruta)} features:\n{features_removed_by_boruta}")
selected = sorted(list(set(boruta_selector.columns_original)))
print("\nList of the features selected by Boruta:")
pprint.pprint(list(selected))
if selected is None:
    raise Error("You need to specify a backup set of variables")

In [ ]:
sample = sample_full.select_features(selected)

## Data Visualization of numerical features and categorical features

In [ ]:
plot_ecdf_df(sample.features, iqr_multiple=IQR_MULTIPLE)

# ModelPipelineDF training


## Grid search and cross validator

We use a custom CV splitter defined in the yield-engine package (see https://scikit-learn.org/stable/glossary.html#term-cv-splitter for the scikit-learn API regarding CV splitter).

In [ ]:
# define the circular cross validator with 30 folds
circular_cv = CircularCrossValidator(test_ratio=1/3, num_splits=3)

The class **ModelPipelineDF** specifies a model as an estimator and a preprocessing pipeline.
The class **ModelGrid** specifies a **ModelPipelineDF**  and a hyperparameter grid.

In [ ]:
def get_model_grids(sample: Sample) -> List[ModelGrid]:
    grids = [ModelGrid(
         model=ModelPipelineDF(predictor=RandomForestRegressor(random_state=42),
                     preprocessing=make_preprocessor(sample=sample, missing=False)),
         estimator_parameters = {"n_estimators": [10],
                                 "max_depth": [4,5,6],
                                 "min_samples_leaf": [4],
                                 "criterion": ["mse"], #["mae","mse"],
                                 "max_features": [1.0]})]
    return grids
grids = get_model_grids(sample)

In [ ]:
ranker = ModelRanker(grids=grids, cv=circular_cv)
ranking = ranker.run(sample, n_jobs=-3)
print(summary_report(ranking))

In [ ]:
top_model = ranking[0]
print(top_model.scoring['test_score'])
print(top_model.parameters)

## ModelPipelineDF inspection
The **PredictorFitCV** summarizes all the information of a model: the estimator used for the model, the CV (=cross-validation) type, and the **Sample** itself.

In [ ]:
predictor = PredictorFitCV(model=top_model.model, cv=circular_cv, sample=sample)
inspector = ModelInspector(predictor)
predictions = predictor.predictions_for_all_splits()

In [ ]:
plt.scatter(sample.target.sort_index(), 
            predictions.groupby(predictions.index)['prediction'].mean().sort_index(), alpha=.3)
plt.xlabel('actual')
plt.ylabel('predicted');

## Shape values

The inspector object allows directly to acces the shap values of the model. These shap values are computed for a given sample as the average of the shap values over all the test folds containg that given sample.

In [ ]:
M = inspector.shap_matrix()
M.head()

In [ ]:
inspector.feature_importances().sort_values(ascending=False).to_frame()

## Shap clustering
The Shap clustering clusters the features using as distance between the features, the correlation matrix of the shap values.
Then using a hierarchical clustering, and visualization style defined in the yield-engine package, one can easily visualize the clustering of the features.

In [ ]:
linkage_tree = inspector.cluster_dependent_features()

In [ ]:
number_features = predictor.sample.features.shape[1]
ax = plt.figure(figsize=(10,number_features*0.5)).add_subplot(111)
style = LineStyle(ax)
DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style).draw()

In [ ]:
number_features = predictor.sample.features.shape[1]
ax = plt.figure(figsize=(10, number_features*.5)).add_subplot(111)
style = FeatMapStyle(ax)
DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style).draw()

It is desirable to have a model with
- good predictivity (good R2 score for instance)
- few features
- independent features  

With the above dendrograms one can isolate **features with low importance and which are strongly realted to other features. It makes sense to discard those.**

# Shap clustering iterations

## Method the run an iteration
Based on the above remarks, who are going to run a clustering iteration as:
1. Based on the shap dendrogram select features to discard
2. Re-run the model with the new set of features
3. Plot again the shap dendrogram to see if the feautures are more independent, and iterate this process if necessary

In [ ]:
def shap_clustering_iteration(sample: Sample, black_list: List[str]) -> Tuple[Sample, ModelInspector]:
    """Removing black list features, and retrain the model. Return a new sample and a fitted model inspector."""
    # Create a new Sample by removing the features in the blacklist
    features = sample.feature_names
    if not set(black_list) <= set(features):
        log.warning(f"""The black list must be a subset of the set of features: the features 
        {set(black_list)-set(features)} are in black list but not in the features.""")

    white_list = sorted(list(set(features) - set(black_list)))
    log.info(f"New white list:\n {white_list}")
    new_sample = sample.select_features(white_list)
    
    # Create the preprocessing pipeline for the model run
    outlier_step = make_outlier_transformer(sample=new_sample, iqr_threshold=IQR_MULTIPLE)
    preprocessor = make_preprocessor(sample=new_sample, missing=True, onehot=True)
    pipeline = PipelineDF(steps = [('outlier_removal', outlier_step), ('preprocess', preprocessor)])
    
    # Run the pipeline
    grids = get_model_grids(new_sample)
    ranker = ModelRanker(grids=grids, cv=circular_cv)
    ranking = ranker.run(new_sample, n_jobs=-3)
    top_model = ranking[0]
    
    # Report the model result
    print(summary_report(ranking))
    print(f"top_model score: {top_model.scoring['test_score']}")
    print(f"top_model parameters: {top_model.parameters}")
    predictor = PredictorFitCV(model=top_model.model, cv=circular_cv, sample=new_sample)
    inspector = ModelInspector(predictor)
    
    return new_sample, inspector

In [ ]:
def plot_shap_dendrogram(inspector: ModelInspector) -> None:
    """Plot dendrogram of the shape clustering."""
    M = inspector.shap_matrix()
    number_features = inspector.model_fit.sample.features.shape[1]
    linkage_tree = inspector.cluster_dependent_features()
    ax = plt.figure(figsize=(10,number_features*.5)).add_subplot(111)
    style = FeatMapStyle(ax)
    DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style).draw();

## Shap culstering iteration 1

In [ ]:
black_list1 = ['WoodDeckSF', 
               #'BsmtFinSF1', 
               "OpenPorchSF",
               #"MSSubClass",
               "KitchenAbvGr",
               'BsmtUnfSF', 
               #'MasVnrArea',
               'LotFrontage']

In [ ]:
sample1, inspector1 = shap_clustering_iteration(sample, black_list1)

In [ ]:
plot_shap_dendrogram(inspector1)

# Shap clustering iteration 2

In [ ]:
black_list2 = [
    "YearRemodAdd",
    "OverallCond",
    "2ndFlrSF",
    #"MSSubClass",
    #"KitchenAbvGr"
]

In [ ]:
sample2, inspector2 = shap_clustering_iteration(sample1, black_list2)

In [ ]:
plot_shap_dendrogram(inspector2)

# Simulation
The simulation builds partial dependency plots which allow to assess the impact thta the value of a given feature has on the model predictions.

In [ ]:
model_fit = inspector2.model_fit
sim = UnivariateSimulator(model_fit=model_fit)

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

dd = widgets.Dropdown(
    options=predictor.sample.features.columns,
    description='Feature:',
    disabled=False,
    layout={"width":"550px"}
)

btn = widgets.Button(description='Simulate')

def plot_simulation(feature:str):
    feature_values = ContinuousRangePartitioning(dataset_df.loc[:,feature]).partitions()
    yield_change = sim.simulate_feature(
            feature_name=feature,
            feature_values=feature_values,
    )
    
    yield_change_aggr = UnivariateSimulator.aggregate_simulation_results(
                    results_per_split=yield_change, percentiles=[10, 50, 90])
    
    XLABEL_TITLE = f"{feature}"
    YLABEL_TITLE = f"Predicted mean yield uplift ({TARGET})"
    COLOR1 = 'red'
    COLOR2 = 'silver'
    
    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(10,10), sharex=True)
    
    # plot lines of prediction
    ax1.set_xlabel(XLABEL_TITLE, color='black', labelpad=10, fontsize=12)
    ax1.set_ylabel(YLABEL_TITLE, color='black', fontsize=12)
    line1, = ax1.plot(yield_change_aggr.index, yield_change_aggr.iloc[:,0], color=COLOR2, linewidth=1)
    line2, = ax1.plot(yield_change_aggr.index, yield_change_aggr.iloc[:,1], color=COLOR1)
    line3, = ax1.plot(yield_change_aggr.index, yield_change_aggr.iloc[:,2], color=COLOR2, linewidth=1)
    ax1.axhline(y=0, color='black', linewidth=.5)
    ax1.tick_params(axis='x', labelcolor='black')
    for pos in ['top', 'right', 'bottom']:
        ax1.spines[pos].set_visible(False)
    ax1.tick_params(axis='x', labelbottom=True, bottom=False)
    ax1.legend((line3, line2, line1), ('90th percentile', 'Median', '10th percentile'), frameon=False)
    
    # plot the histogram
    x = sample.features[feature].dropna()
    hist_range = (min(yield_change_aggr.index), max(yield_change_aggr.index))
    n, bins, patches = ax2.hist(x, edgecolor='white', color=COLOR2, range=hist_range)
    bins1 = pd.Series(bins).rolling(window=2).mean().shift(-1).dropna()
    ax2.invert_yaxis()
    ax2.tick_params(axis='y', labelcolor='black')
    max_y = max(n)
    y_offset = max_y * 0.05
    for (x,y) in zip(bins1, n):
        if y>0:
            ax2.text(x, y + y_offset, str(int(y)), color='black', horizontalalignment='center')
    ax2.get_yaxis().set_visible(False)
    ax2.get_xaxis().set_visible(False)
    for pos in ['top', 'right', 'left', 'bottom']:
        ax2.spines[pos].set_visible(False)
    plt.subplots_adjust(hspace=.2)
    plt.show()

def on_click(btn):
    clear_output()
    display(widgets.HBox([dd, btn]))
    plot_simulation(feature=dd.value)
    
btn.on_click(on_click)    
display(widgets.HBox([dd, btn]))